In [142]:
import pandas as pd
import json
from tqdm import tqdm
import seaborn as sns
from thefuzz import fuzz, process

In [112]:
import pandas as pd
import json

# Load your JSON file
with open("../data/drug-ndc-0001-of-0001 2.json", "r") as file:
    data = json.load(file)

# Use json_normalize to flatten and load the data into a DataFrame
df = pd.json_normalize(data)
ndc = pd.DataFrame(data["results"])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108472ce0>>
Traceback (most recent call last):
  File "/Users/guillaumebelissent/opt/anaconda3/envs/aada/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [3]:
drugbank = pd.read_xml("../data/fulldatabase.xml")

In [210]:
drugbank_ids = pd.read_csv(
    "/Users/guillaumebelissent/Docs/EPFL/MA/ADA/BindingDB_All.tsv",
    sep="\t",
    usecols=["DrugBank ID of Ligand"],
)

/var/folders/6_/6g10b43x17ngwmk4l9msb2mc0000gn/T/ipykernel_23691/2787509164.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  drugbank_ids=pd.read_csv(


In [82]:
medicaid = pd.read_csv(
    "/Users/guillaumebelissent/Docs/EPFL/MA/ADA/ada-2024-project-standarddeviants/data/data-11-08-2024-12_34pm.csv"
)
medicaid["Product Name"] = medicaid["Product Name"].apply(lower)
presrciptions = (
    medicaid.dropna(subset="Number of Prescriptions")
    .groupby("Product Name")[
        ["Number of Prescriptions", "Units Reimbursed", "Total Amount Reimbursed"]
    ]
    .apply(sum)
    .reset_index()
)

/var/folders/6_/6g10b43x17ngwmk4l9msb2mc0000gn/T/ipykernel_23691/1119521177.py:3: FutureWarning: The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.
  presrciptions = medicaid.dropna(subset='Number of Prescriptions').groupby('Product Name')[['Number of Prescriptions','Units Reimbursed','Total Amount Reimbursed']].apply(sum).reset_index()
/Users/guillaumebelissent/opt/anaconda3/envs/aada/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [104]:
drugbank["name"] = drugbank["name"].apply(lower)
ndc["generic_name"] = ndc["generic_name"].apply(lower)
ndc["brand_name"] = ndc["brand_name"].apply(lower)
ndc_db_df = pd.merge(
    drugbank, ndc, left_on="name", right_on="generic_name", how="inner"
)
converter = pd.merge(
    ndc_db_df,
    drugbank_ids,
    left_on="drugbank-id",
    right_on="DrugBank ID of Ligand",
    how="inner",
)[["name", "drugbank-id", "DrugBank ID of Ligand", "brand_name", "generic_name"]]
final = pd.merge(
    converter, presrciptions, left_on="brand_name", right_on="Product Name", how="inner"
).drop_duplicates()

In [105]:
print(f'Merge between NDC and Drugbank: {ndc_db_df["brand_name"].unique().shape[0]}')
print(
    f'Merge between NDC + Drugbank and BindingDB: {converter["brand_name"].unique().shape[0]}'
)
print(f'Merge between all and Medicaid: {final["brand_name"].unique().shape[0]}')

Merge between NDC and Drugbank: 12235
Merge between NDC + Drugbank and BindingDB: 620
Merge between all and Medicaid: 12


In [117]:
ndc[["brand_name", "brand_name_base", "generic_name"]].drop_duplicates()

,brand_name,brand_name_base,generic_name
0,lurasidone hydrochloride,lurasidone hydrochloride,lurasidone hydrochloride
1,Plerixafor,Plerixafor,Plerixafor
2,vilazodone hydrochloride,vilazodone hydrochloride,vilazodone hydrochloride
3,PHENYLEPHRINE HYDROCHLORIDE,PHENYLEPHRINE HYDROCHLORIDE,PHENYLEPHRINE HYDROCHLORIDE
4,The Ordinary Salicylic Acid 0.5%,The Ordinary Salicylic Acid 0.5%,Salicylic Acid
...,...,...,...
138885,Severe Congestion and Pain,Severe Congestion and Pain,"ACETAMINOPHEN, GUAIFENESIN, and PHENYLEPHRINE ..."
138886,HEAD CONGESTION PLUS FLU SEVERE PE,HEAD CONGESTION PLUS FLU SEVERE PE,"ACETAMINOPHEN, DEXTROMETHORPHAN HYDROBROMIDE, ..."
138892,careone allergy relief,careone allergy relief,cetirizine hydrochloride
138893,careone tussin dm,careone tussin dm,"dextromethorphan hydrobromide, guaifenesin"


In [204]:
drugbank_bdb = pd.merge(
    drugbank,
    drugbank_ids,
    left_on="drugbank-id",
    right_on="DrugBank ID of Ligand",
    how="inner",
)[["name", "drugbank-id", "DrugBank ID of Ligand"]].drop_duplicates()
converter = fuzzy_merge(
    drugbank_bdb,
    ndc[["brand_name", "brand_name_base", "generic_name"]].drop_duplicates(),
    left_on="name",
    right_on="generic_name",
    how="inner",
)
presrciptions["Product Name"] = presrciptions["Product Name"].astype(str)
converter["brand_name"] = converter["brand_name"].astype(str)
final = fuzzy_merge(
    converter, presrciptions, left_on="brand_name", right_on="Product Name", how="inner"
).drop_duplicates()

100%|██████████| 13678/13678 [04:32<00:00, 50.27it/s] 


In [195]:
print(
    f'Merge between BindingDB and Drugbank: {drugbank_bdb["drugbank-id"].unique().shape[0]}'
)
print(
    f'Merge between NDC and Drugbank + BindingDB: {converter["drugbank-id"].unique().shape[0]}'
)
print(
    f'Merge between all and Medicaid: {final.dropna()["drugbank-id"].unique().shape[0]}'
)

Merge between BindingDB and Drugbank: 1251
Merge between NDC and Drugbank + BindingDB: 1251
Merge between all and Medicaid: 980


In [207]:
text_cols = [
    "name",
    "fuzzy_match_name",
    "brand_name",
    "brand_name_base",
    "generic_name",
    "fuzzy_match_brand_name",
    "Product Name",
]
for col in text_cols:
    final.loc[:, col] = final.loc[:, col].apply(lower)

In [209]:
final.to_csv("prescription_data.csv")

In [206]:
final.drop_duplicates()

,name,drugbank-id,DrugBank ID of Ligand,fuzzy_match_name,brand_name,brand_name_base,generic_name,fuzzy_match_brand_name,Product Name,Number of Prescriptions,Units Reimbursed,Total Amount Reimbursed
0,paroxetine,DB00715,DB00715,paroxetine,paroxetine,paroxetine,paroxetine,p,p,12.0,20.000,7.62
2,ergometrine,DB01253,DB01253,methylergometrine maleate,nan,none,methylergometrine maleate,12 hour na,12 hour na,8085.0,229998.600,125032.85
3,dasatinib,DB01254,DB01254,dasatinib monohydrate,nan,none,dasatinib monohydrate,12 hour na,12 hour na,8085.0,229998.600,125032.85
4,decitabine,DB01262,DB01262,decitabine,nan,none,decitabine,12 hour na,12 hour na,8085.0,229998.600,125032.85
5,decitabine,DB01262,DB01262,decitabine,decitabine,decitabine,decitabine,decitabine,decitabine,3125.0,4879.132,195089.59
...,...,...,...,...,...,...,...,...,...,...,...,...
13673,silibinin,DB09298,DB09298,tin,stannum metallicum,stannum metallicum,tin,kanuma,kanuma,216.0,8980.000,9369008.79
13674,sulbactam,DB09324,DB09324,"ampicillin sodium, sulbactam sodium",ampicillin sodium and sulbactam sodium,ampicillin sodium and sulbactam sodium,"ampicillin sodium, sulbactam sodium",ampicillin,ampicillin,63309.0,1018973.073,5478581.43
13675,osimertinib,DB09330,DB09330,osimertinib,tagrisso,tagrisso,osimertinib,tagrisso,tagrisso,1626.0,50860.000,27485107.70
13676,rolofylline,DB05360,DB05360,doxofylline,nan,none,doxofylline,12 hour na,12 hour na,8085.0,229998.600,125032.85


In [139]:
"amoxicillin" in final["generic_name"].to_list()

False

In [14]:
converter

,name,drugbank-id,DrugBank ID of Ligand,brand_name,generic_name
0,paroxetine,DB00715,DB00715,paroxetine,paroxetine
1,paroxetine,DB00715,DB00715,paroxetine,paroxetine
2,paroxetine,DB00715,DB00715,paroxetine,paroxetine
3,paroxetine,DB00715,DB00715,paroxetine,paroxetine
4,paroxetine,DB00715,DB00715,paroxetine,paroxetine
...,...,...,...,...,...
191605,osimertinib,DB09330,DB09330,NaN,osimertinib
191606,osimertinib,DB09330,DB09330,NaN,osimertinib
191607,osimertinib,DB09330,DB09330,NaN,osimertinib
191608,osimertinib,DB09330,DB09330,NaN,osimertinib


In [15]:
ndc_db_df["name"].unique().shape

(2109,)

In [16]:
converter["name"].unique().shape

(125,)

In [58]:
converter.drop_duplicates().dropna()

,drugbank-id,DrugBank ID of Ligand,brand_name,generic_name
0,DB00715,DB00715,Paroxetine,Paroxetine
11687,DB01254,DB01254,Dasatinib,Dasatinib
23713,DB01262,DB01262,Decitabine,Decitabine
23897,DB01263,DB01263,Posaconazole,Posaconazole
23923,DB01263,DB01263,POSACONAZOLE,Posaconazole
...,...,...,...,...
84255,DB09085,DB09085,Tetracaine,Tetracaine
84375,DB09086,DB09086,AnaDent Childrens Kanka Gel,Eugenol
84397,DB09119,DB09119,Eslicarbazepine acetate,Eslicarbazepine acetate
84405,DB09128,DB09128,Brexpiprazole,Brexpiprazole


In [8]:
medicaid = pd.read_csv(
    "/Users/guillaumebelissent/Docs/EPFL/MA/ADA/ada-2024-project-standarddeviants/data/data-11-08-2024-12_34pm.csv"
)

In [38]:
pd.merge(
    converter, presrciptions, left_on="brand_name", right_on="Product Name", how="inner"
).drop_duplicates()

,name,drugbank-id,DrugBank ID of Ligand,brand_name,generic_name,Product Name,Number of Prescriptions,Units Reimbursed,Total Amount Reimbursed
0,paroxetine,DB00715,DB00715,paroxetine,paroxetine,paroxetine,1195464.0,5.016026e+07,15493652.44
12388,decitabine,DB01262,DB01262,decitabine,decitabine,decitabine,3125.0,4.879132e+03,195089.59
12584,pretomanid,DB05154,DB05154,pretomanid,pretomanid,pretomanid,45.0,1.154000e+03,26430.45
12585,alogliptin,DB06203,DB06203,alogliptin,alogliptin,alogliptin,150656.0,7.406709e+06,34548183.73
12970,lacosamide,DB06218,DB06218,lacosamide,lacosamide,lacosamide,594771.0,9.548880e+07,28793335.19
12995,lacosamide,DB06218,DB06218,motpoly xr,lacosamide,motpoly xr,12.0,6.560000e+02,13048.52
16145,etravirine,DB07191,DB07191,etravirine,etravirine,etravirine,1987.0,1.326700e+05,1829000.46
17801,dabigatran etexilate,DB06695,DB06695,dabigatran,dabigatran etexilate,dabigatran,7892.0,4.969630e+05,1155960.10
17813,plerixafor,DB06809,DB06809,plerixafor,plerixafor,plerixafor,68.0,6.766800e+02,149325.59
18352,fingolimod,DB08868,DB08868,fingolimod,fingolimod,fingolimod,7168.0,2.273370e+05,8755937.29


In [33]:
sum(medicaid["Number of Prescriptions"].dropna())

690392512.0

In [35]:
medicaid[["Product Name", "Number of Prescriptions"]]

,Product Name,Number of Prescriptions
0,TRULICITY,111.0
1,TRULICITY,127.0
2,EMGALITY P,28.0
3,TALTZ AUTO,30.0
4,TALTZ AUTO,NaN
...,...,...
2550438,MOVANTIK 2,NaN
2550439,IZERVAY,NaN
2550440,SEMGLEE IN,NaN
2550441,INSULIN GL,NaN


In [37]:
medicaid[["Product Name", "Number of Prescriptions"]].sort_values(
    "Number of Prescriptions"
).drop_duplicates()

,Product Name,Number of Prescriptions
986186,PHENAZOPYR,11.0
501259,TETRABENAZ,11.0
501249,BUPROPION,11.0
501217,DESVENLAFA,11.0
2310220,HYDRALAZIN,11.0
...,...,...
2518566,GS CHLD CO,NaN
2523016,XOPENEX 1.,NaN
2529272,QC CHILD A,NaN
2530219,VENTAVIS 2,NaN


In [41]:
medicaid["Product Name"] = medicaid["Product Name"].apply(lower)

NameError: name 'lower' is not defined

In [62]:
medicaid["Product Name"].unique().shape

(11073,)